# Solving the MountainCar problem

In this task we have a car, located at the bottom of a valley. The car can go to left or right with full throttle or do nothing.
Here the environment is a bit changed because the reward function is redefined to provide a dense feedback. In the original game -1 reward is received all the time. However this makes the convergence slow. 

To speed it up, the reward function will be:

$$r = \sin(p' \cdot 3) \cdot 0.45 + 0.55$$

where $p'$ is the position in the next state.

**State:** the $x$ coordinate of the car and the speed

**Action:** full throttle to left or right (left: 0, do nothing: 1 \[we will ignore this\], right: 2)

The solution will use the radial basis functions for feature extraction. You will implement Q-learning with a linear function approximator (based on the features) to train the Q-function.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import gym  # this library contains environments defined in advance

<img src="http://drive.google.com/uc?export=view&id=1XJj3Bju-mqZO8S9JT9QEfMAnZqAjj5X2" width=45%>

### Feature extraction

In [ ]:
def init_weights(num_coarse):
    # num_coarse - how coarse is the feature space
    # the Q-function will be represented by a linear-approximator
    # initialize the weights matrix 
    # each weight correpsonds to a specific feature
    
    # ----- implement this single line -----
    return   # create a numpy matrix with the appropriate size, now we have only two actions!


def center_points(num_coarse):
    # in case of the radial functions we need center points
    # the state space is 2D
    max_speed = 0.07
    min_speed = -0.07
    max_pos = 0.6
    min_pos = -1.2
    speeds = np.linspace(min_speed, max_speed, num_coarse)  
    positions = np.linspace(min_pos, max_pos, num_coarse)
    sigma = (max_pos - min_pos) / num_coarse
    ratio = (max_pos - min_pos) / (max_speed - min_speed)  # the scale of the speed and position is different, this would harm the exponential terms calculated from the speed, causing no effect of the speed
    return positions, speeds, sigma, ratio


def feature(state, centers):
    # calculate the feature values
    # take into account the ratio too
    pos, velocity = state  # position, velocity
    positions, speeds, sigma, ratio = centers
    
    # ----- implement the radial basis features according to the formula -----
    
    
    return f  # the feature values in a numpy matrix

In [ ]:
# testing the feature function
# this is a visualization to understand better the feature space
# the feature hashigh value at the location of the state in the state space
# the state space is spanned by the position and velocity of the car
# therefore the state space is 2-dimensional
cps = center_points(5)

state = (0.3, 0.04)
f = feature(state, cps)
plt.matshow(f)

state = (-0.3, -0.04)
f = feature(state, cps)
plt.matshow(f)

state = (0.1, 0.01)
f = feature(state, cps)
plt.matshow(f)

You should see for the first case:

<img src="http://drive.google.com/uc?export=view&id=1WyxbvdQm43yxuTRARRRpV5PtxwmF950G" width=45%>

### Saving/loading weights

In [ ]:
# At the end we can save the weights.
# This is good for experimenting with the agent after training.

def save_weights(weights, file_path):
    np.save(file_path, weights, allow_pickle=False)


def load_weights(file_path):
    return np.load(file_path, allow_pickle=False)

### Functions for training

In [ ]:
def Q_sa(state, action, weights, centers):
    # this is the Q-function
    # ----- implement this -----
    f =   # calculate the features for the current state
    return   # calculate the Q-value from the weights, features and action (action: 0, 1)


def policy(state, weights, centers, epsilon):
    # epsilon-greedy for choosing the next action
    # return 0 for left, 2 for right (1 is ignored now)
    # ----- implement this -----
    
    
    
    
    return action  # the sampled action


def update(state, action, r, state_next, weights, centers, gamma, alpha):
    # Q-learning update rule for the linear approximation
    # ----- implement this -----
    q =   # choose the best Q-value
    u =   # one-step return
    action = action // 2  # action can be 0 or 2 but 2 is not a valid index
    weights[action] +=   # update the weights

### Training the agent

In [ ]:
def train():
    # init the environment
    env = gym.make('MountainCar-v0')
    # set the seeds to make the results easier to reproduce
    np.random.seed(0)
    env.seed(1)
    
    # hyper-parameters (you can play with this parameters)
    num_iter = 5000
    num_coarse = 10
    epsilon = 1
    alpha = 0.35  # learning rate
    gamma = 0.98
    
    weights = init_weights(num_coarse)
    centers = center_points(num_coarse)

    for counter in range(num_iter):
        state = env.reset()  # state - [position, velocity]
        done = False
        score = 0  # to see the return (for status log)
        length = 0  # the length of the current episode (for status log)
        max_pos = -1.2  # how much further moved the car in the right direction (for status log)
        while not done:
            length += 1
            action =   # ---- get the action in the current state -----
            state_next, _, done, _ =   # ----- make one step with the environment -----
            if state_next[0] > max_pos:
                max_pos = state_next[0]  # this will show whether the agent is getting closer to the flag
            reward = np.sin(state_next[0]*3)*0.45+0.55  # redefined reward to get more feedback
            update(state, action, reward, state_next, weights, centers, gamma, alpha)
            score += reward
            state = state_next
        epsilon = max(epsilon - 1.0/num_iter , 0.1)  # decrease the epsilon (exploration -> exploitation)
        if counter % 10 == 0:  # logging
            print("{} -> {} -> {} -> {}".format(counter, length,  score, max_pos))
    # saving the weights at the end
    save_weights(weights, "wgt.npy")

### Video for seeing what the agent is doing

In [ ]:
def visualize(weights, num_coarse):
    env = gym.make('MountainCar-v0')
    centers = centers = center_points(num_coarse)
    for counter in range(10):
        done = False
        state = env.reset()  # state - [position, velocity]
        score = 0
        while not done:
            action = policy(state, weights, centers, 0.0)  # during test there is no need for exploration
            state, reward, done, _ = env.step(action)
            env.render()  # to show a window with the moving car
            score += reward
        print("At iteration {} the score: {}".format(counter, score))

In [ ]:
# This will not work on colab but it can run on a common laptop
# No need for gpu and computationally not expensive
weights = load_weights("wgt.npy")
num_coarse = 10
visualize(weights, num_coarse)